In [ ]:
# importing the data stream
# %%
import mne

sample_dir = mne.datasets.sample.data_path()
sample_fname = sample_dir / 'MEG' / 'sample' / 'sample_audvis_raw.fif'

raw = mne.io.read_raw_fif(sample_fname)
raw.crop(tmax=60)

events = mne.find_events(raw, stim_channel='STI 014')
event_id = {'auditory/left': 1, 'auditory/right': 2, 'visual/left': 3,
            'visual/right': 4, 'face': 5, 'buttonpress': 32}

epochs = mne.Epochs(raw, events=events, event_id=event_id,
                    tmin=-0.2, tmax=0.5, baseline=(None, 0),
                    preload=True)
import numpy as np
import pandas as pd

data = pd.read_csv("fNIRS_FingerTapping.csv")
# Transpose the DataFrame
data_transposed = data.T

# Reset the index if necessary (optional)
data = data_transposed.reset_index(drop=True).values

ch_names = [
    "S1_D1 hbo",
    "S1_D1 hbr",
    "S2_D1 hbo",
    "S2_D1 hbr",
]
ch_types = [
    "hbo",
    "hbr",
    "hbo",
    "hbr",
]
sfreq = 10.0  # in Hz

info = mne.create_info(ch_names=ch_names, ch_types=ch_types, sfreq=sfreq)
raw = mne.io.RawArray(data, info, verbose=True)

montage = mne.channels.make_standard_montage("artinis-octamon")
raw.set_montage(montage)

# View the position of optodes in 2D to confirm the positions are correct.
#raw.plot_sensors()

subjects_dir = mne.datasets.sample.data_path() / "subjects"
mne.datasets.fetch_fsaverage(subjects_dir=subjects_dir)

#brain = mne.viz.Brain(
#    "fsaverage", subjects_dir=subjects_dir, alpha=0.5, cortex="low_contrast"
#)
#brain.add_head()
#brain.add_sensors(raw.info, trans="fsaverage")
#brain.show_view(azimuth=90, elevation=90, distance=500)

In [119]:
# Running the tests on the data
raw.annotations.set_durations(5)
unwanted = np.nonzero(raw.annotations.description == "15.0")
raw.annotations.delete(unwanted)

In [ ]:
picks = mne.pick_types(raw.info, meg=False, fnirs=True)
dists = mne.preprocessing.nirs.source_detector_distances(
    raw.info, picks=picks
)
raw.pick(picks[dists > 0.01])
raw.plot(
    n_channels=len(raw.ch_names), duration=500, show_scrollbars=False
)

In [ ]:
raw_od = mne.preprocessing.nirs.optical_density(raw, verbose=False)
raw_od.plot(n_channels=len(raw.ch_names), duration=500, show_scrollbars=False)
# evaluating data quality
sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)
fig, ax = plt.subplots(layout="constrained")
ax.hist(sci)
ax.set(xlabel="Scalp Coupling Index", ylabel="Count", xlim=[0, 1])

In [ ]:
raw_od.info["bads"] = list(compress(raw_od.ch_names, sci < 0.5))
#coverting it to haemodynamic values
raw_haemo = mne.preprocessing.nirs.beer_lambert_law(raw_od, ppf=0.1)
raw_haemo.plot(n_channels=len(raw_haemo.ch_names), duration=500, show_scrollbars=False)

In [ ]:
#removing the hear rate from the signal
raw_haemo_unfiltered = raw_haemo.copy()
raw_haemo.filter(0.05, 0.7, h_trans_bandwidth=0.2, l_trans_bandwidth=0.02)
for when, _raw in dict(Before=raw_haemo_unfiltered, After=raw_haemo).items():
    fig = _raw.compute_psd().plot(
        average=True, amplitude=False, picks="data", exclude="bads"
    )
    fig.suptitle(f"{when} filtering", weight="bold", size="x-large")


In [ ]:
events, event_dict = mne.events_from_annotations(raw_haemo)
reject_criteria = dict(hbo=80e-6)
tmin, tmax = -5, 15

epochs = mne.Epochs(
    raw_haemo,
    events,
    event_id=event_dict,
    tmin=tmin,
    tmax=tmax,
    reject=reject_criteria,
    reject_by_annotation=True,
    proj=True,
    baseline=(None, 0),
    preload=True,
    detrend=None,
    verbose=True,
)
epochs.plot_drop_log()
epochs.plot_image(
    combine="mean",
    vmin=-30,
    vmax=30,
    ts_args=dict(ylim=dict(hbo=[-15, 15], hbr=[-15, 15])),
)


In [ ]:
# plotting the standard fNIR graph
evoked_dict = {
    "HbO": epochs.average(picks="hbo"),
    "HbR": epochs.average(picks="hbr"),
}

# Rename channels until the encoding of frequency in ch_name is fixed
for condition in evoked_dict:
    evoked_dict[condition].rename_channels(lambda x: x[:-4])
 
color_dict = dict(HbO="#AA3377", HbR="b")
styles_dict = dict(Control=dict(linestyle="dashed"))

mne.viz.plot_compare_evokeds(
    evoked_dict, combine="mean", ci=0.95, colors=color_dict, styles=styles_dict
)